In [62]:
import pandas as pd
import numpy as np

import json
import spacy

In [43]:
pd.set_option('display.max_rows', 200_000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 2_500)

In [44]:
files = ['debate1.txt', 'debate2.txt', 'debate3.txt', 'debate4.txt', 'debate5.txt']

data = []
for file in files:
    for line in open(file, 'r'):
        if line != '\n':
            data.append(json.loads(line))

In [45]:
len(data)

185029

In [46]:
data = [line for line in data if 'created_at' in line]

In [47]:
len(data)

180490

In [51]:
tweets_list = []
years = []
months = []
days = []
hours = []
minutes = []
seconds = []
datetime = []

for tweet in data:
    date = pd.Timestamp(tweet['created_at'])
    years.append(date.year)
    months.append(date.month)
    days.append(date.day)
    hours.append(date.hour - 3)
    minutes.append(date.minute)
    seconds.append(date.second)
    datetime.append(pd.Timestamp(year=date.year, month=date.month, day=date.day, hour=date.hour - 3, minute=date.minute, second=date.second))
    try:
        try:
            if tweet['text'].startswith('RT'):
                try:
                    tweets_list.append(tweet['retweeted_status']['extended_tweet']['full_text'])
                except:
                    tweets_list.append(tweet['retweeted_status']['text'])
            else:
                tweets_list.append(tweet['text'])
        except:
            tweets_list.append(tweet['extended_tweet']['full_text'])
    except:
        tweets_list.append(tweet['text'])

In [52]:
tweets = pd.DataFrame({'text': tweets_list, 'hour': hours, 'minute': minutes, 'second': seconds, 'day': days, 'month': months, 'year': years, 'datetime': datetime})

tweets.head()

,text,hour,minute,second,day,month,year,datetime
0,Gente vamo assistir o debate,17,46,30,26,9,2018,2018-09-26 17:46:30
1,"Acompanhe o debate entre os presidenciáveis promovido pelo UOL, Folha e SBT #UOLnasUrnas #DebateSBT https://t.co/qENAPIsFp6",17,46,30,26,9,2018,2018-09-26 17:46:30
2,não vou me estressar com politica hoje\n\nbolsominion: a\n\neu: https://t.co/AZkQNR0k8D,17,46,30,26,9,2018,2018-09-26 17:46:30
3,Alerta: debate no SBT começando agora às 17:45 e temos DACIOLO!,17,46,30,26,9,2018,2018-09-26 17:46:30
4,Assista ao debate entre candidatos à Presidência da República https://t.co/yblVx0h25i,17,46,30,26,9,2018,2018-09-26 17:46:30


In [53]:
tweets.shape

(180490, 8)

In [54]:
#tweets.to_csv('tweets_debate.csv', index=False)

In [55]:
from spacy.lang.pt.stop_words import STOP_WORDS

D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [63]:
nlp = spacy.load('pt')

In [79]:
import re

In [59]:
import string
SYMBOLS = ' '.join(string.punctuation).split(' ') + ['-', '...', '”', '”']

In [92]:
def normalizar(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    clean_comment = re.sub(r'\bhttps://t\.co/\w+\b', '', comment)
    clean_comment = nlp(clean_comment)
    lemmatized = list()
    for word in clean_comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and ((lemma not in STOP_WORDS) and (lemma not in SYMBOLS))):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

In [ ]:
tweets['text_clean'] = tweets['text'].apply(normalizar, lowercase=True, remove_stopwords=True)

In [ ]:
tweets_shuffled = tweets.sample(frac=1).reset_index(drop=True)

In [ ]:
#tweets_shuffled.to_csv('tweets_debate_limpo.csv', index=False)

In [33]:
tweets_clean = pd.read_csv('tweets_debate_limpo.csv')

tweets_clean.head()

,text,hour,minute,second,day,month,year,datetime,text_clean
0,"""foda-se a eleição, eu quero ver o jogo porrraaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"" - Álvaro Dias",19,24,23,26,9,2018,2018-09-26 19:24:23,""" foder - o eleição , querer o jogar porrraaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa "" - álvaro"
1,MANO O DACIOLO É FODA GLÓRIA DEUS,19,12,45,26,9,2018,2018-09-26 19:12:45,manir o daciolo foder glória deus
2,"Porra Daciolo eu tinha até me emocionado com a declaração pra mamãe e esposa, agora tu mete essa de voto impresso meu consagrado? #DebateSBT",19,13,41,26,9,2018,2018-09-26 19:13:41,"porra daciolo emocionar o declaração pra mamãe e esposo , meter votar impresso consagrar ? # debatesbt"
3,"Daciolo falou pouco, mas falou bonito...",18,37,45,26,9,2018,2018-09-26 18:37:45,"daciolo falar , falar bonito ..."
4,"@morgalashnikova Eu n valorizei a grandeza desse candidato antes, mas agora eu percebi meu engano. 51 é só coisa boa, pinga e Daciolo",19,12,20,26,9,2018,2018-09-26 19:12:20,"@morgalashnikova n valorizar o grandeza d candidatar antar , perceber enganir . 51 só , pingar e daciolo"


In [95]:
tweets_clean['text_clean'] = tweets_clean['text'].apply(normalizar, lowercase=True, remove_stopwords=True)

In [96]:
sentiments = [0, 1, -1, 1, 1, 0, 1, 1, 1, -1, #0xx
              1, -1, 1, -1, 0, 1, 1, 1, 1, 0,
              -1, 1, 1, 0, 1, 0, 1, -1, 1, 1,
              1, 1, 1, 0, 0, -1, 1, 0, -1, 0,
              1, 0, 1, -1, 1, 1, 1, 1, 1, -1,
              1, 1, 0, -1, -1, 0, -1, 1, 0, 0,
              1, 1, 1, 0, -1, 0, -1, -1, 0, 0,
              0, 0, 1, 1, 1, -1, 0, -1, -1, 0,
              1, 0, 0, 0, 1, -1, -1, -1, -1, 0,
              -1, 1, 0, -1, 0, 1, 0, 1, 1, -1,
              1, -1, 1, -1, 0, -1, -1, -1, -1, 1, #1xx
              0, -1, 1, 1, -1, 1, 0, -1, 0, 1,
              1, 0, -1, 1, 0, -1, 1, 1, 1, 0,
              0, 1, -1, 0, -1, 1, 0, -1, -1, -1,
              0, -1, 1, 0, 1, -1, -1, 1, 1, 1,
              -1, 0, 0, 1, -1, 0, 1, 0, -1, 0,
              -1, 1, 0, 1, 0, 1, 1, -1, 0, 1,
              0, -1, 1, 0, 0, 1, -1, 1, 0, 0,
              1, 1, 0, 0, 1, 1, 1, 0, -1, 0,
              -1, -1, 1, 1, -1, 1, 0, 0, 0, 1,
              0, 0, 1, 1, -1, 1, 0, -1, 1, 1,      #2xx
              1, 1, -1, 0, 1, 0, 1, 0, 0, 0,
              1, 0, -1, 0, 0, 0, 1, 0, 1, -1,
              0, 0, 1, -1, 0, 1, 0, 0, 0, 1,
              1, 0, 1, 0, 1, 0, -1, 1, 1, -1,
              1, -1, 0, 0, 0, 0, 1, 1, 0, 1,
              1, -1, 0, 1, 0, -1, -1, 0, -1, 1,
              1, -1, 0, -1, 0, -1, -1, -1, 1, 0,
              0, 0, -1, 1, -1, -1, 0, 1, -1, -1,
              -1, 1, 0, 1, 0, 1, 1, 0, -1, 1,
              0, 1, 1, 0, -1, 1, 1, 0, 1, 0,   #3xx
              1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
              1, 0, -1, 0, 1, 0, -1, 0, 0, -1,
              1, 1, 1, 1, 1, 1, 0, -1, -1, -1,
              0, -1, 0, -1, -1, -1, -1, 1, 1, -1,
              -1, 0, 1, 1, 0, 1, 1, 0, 1, -1,
              1, 1, -1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, -1, 1, 1, 0, 0, 0, 0, 1,
              -1, 0, 0, -1, -1, 0, 0, 1, -1, 0,
              1, 1, 1, 1, 0, 1, 0, -1, -1, -1,
              1, -1, 0, 1, -1, -1, 1, 1, -1, 1,  #4xx
              -1, 1, 1, -1, 1, 1, -1, -1, 1, 1,
              -1, 0, 1, -1, 1, 1, 0, 1, 1, 0,
              0, -1, 1, -1, -1, 1, -1, 0, 1, -1,
              0, 1, -1, 0, 1, 1, 0, 1, 0, 0,
              -1, 0, 1, 1, 1, 1, 0, -1, 1, 1,
              -1, 0, 1, -1, 0, 0, -1, 0, 1, 1,
              1, 1, 0, 1, 0, 0, -1, -1, -1, -1,
              0, -1, 1, 1, 0, 0, -1, 1, -1, 1,
              0, -1, -1, -1, -1, 0, 1, 0, -1, -1,
              -1, -1, -1, 0, -1, 0, 0, 0, 1, 1,  #5xx
              -1, 1, 1, -1, -1, 1, 1, 0, 1, -1, 
              1, 0, 0, 1, 1, 1, 0, 1, 0, -1,
              1, 0, 1, -1, -1, -1, 1, 1, 1, 0,
              0, 1, -1, 1, -1, 0, 0, 1, 1, -1,
              -1, 0, 0, -1, 1, 0, 1, 0, 1, 1,
              0, 1, -1, 1, 1, 0, 0, -1, 1, -1,
              -1, -1, -1, 0, -1, 1, 1, -1, 1, 1,
              -1, -1, -1, 1, -1, 1, -1, -1, 0, -1,
              0, 1, 0, 1, 1, 0, 1, 1, -1, 0,
              1, 1, -1, 1, 1, 1, 1, 1, 1, -1,  #6xx
              0, 1, -1, 1, -1, 1, 1, 1, 0, 0,
              1, -1, -1, 1, -1, -1, -1, -1, -1, 1,
              -1, -1, 1, 0, -1, 0, 1, -1, 1, 1,
              -1, 1, -1, -1, 1, 0, 1, 1, -1, -1,
              1, -1, 1, -1, -1, 1, 0, -1, 1, 1,
              1, -1, 1, 0, 0, -1, -1, 1, 1, 0,
              1, 1, 1, 1, 1, 1, 0, -1, 0, -1,
              -1, -1, -1, -1, 0, -1, 0, 1, 1, 0,
              -1, 0, 1, 1, 0, 0, -1, -1, 0, 0,
              -1, 1, 0, 1, -1, 1, 1, 0, 0, 1, #7xx
              1, 0, 1, -1, 1, 0, 1, 1, 1, 1,
              -1, -1, -1, -1, 1, -1, 1, 1, 1, 1,
              1, 0, -1, 0, 0, 0, 1, 1, -1, 0,
              1, 1, 0, -1, 0, 1, 1, 1, -1, 1,
              -1, -1, 1, 0, 1, 0, -1, 1, 0, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
              0, 1, 1, 1, 1, 1, -1, 1, 0, -1,
              -1, -1, -1, 0, -1, 1, 0, -1, 0, 1,
              1, 0, -1, 1, -1, -1, 0, -1, -1, -1,
              1, -1, 0, 1, 1, 1, 1, 0, 0, -1,  #8xx
              1, 1, -1, -1, 0, 0, -1, 1, -1, -1,
              -1, 1, -1, 1, 1, 0, 0, -1, 0, 1,
              0, 1, 1, 0, 1, -1, -1, -1, 1, -1,
              1, 1, -1, 1, 1, 0, -1, -1, 0, 0,
              1, -1, 1, 0, 1, 1, -1, 1, -1, -1,
              -1, -1, 0, -1, 1, 0, 1, 0, 0, 1,
              0, 1, -1, 1, 0, 1, -1, 0, -1, 0,
              1, 0, -1, 1, -1, 1, 1, 1, 1, 0,
              -1, 0, 0, -1, -1, -1, 1, 1, -1, 1,
              -1, 0, 1, 0, -1, 1, 1, 1, 1, 1,  #9xx
              -1, 0, 0, 0, -1, -1, -1, 1, -1, 1,
              1, -1, 0, -1, 0, 0, 1, -1, 0, -1,
              -1, 1, -1, -1, -1, 1, 0, -1, 0, 1,
              1, 1, 0, 1, -1, 1, 0, -1, 1, 0,
              -1, 1, 1, 0, 0, 1, 1, 1, 0, -1,
              -1, 0, 0, 1, -1, -1, 1, 0, 0, 1,
              0, -1, 0, -1, 1, -1, 1, -1, 0, -1,
              -1, -1, 1, -1, 0, 1, 1, 1, -1, 1,
              -1, 1, -1, 1, 0, 0, 0, -1, -1, -1]

In [97]:
sentiments = pd.Series(sentiments)

sentiments

0      0
1      1
2     -1
3      1
4      1
5      0
6      1
7      1
8      1
9     -1
10     1
11    -1
12     1
13    -1
14     0
15     1
16     1
17     1
18     1
19     0
20    -1
21     1
22     1
23     0
24     1
25     0
26     1
27    -1
28     1
29     1
30     1
31     1
32     1
33     0
34     0
35    -1
36     1
37     0
38    -1
39     0
40     1
41     0
42     1
43    -1
44     1
45     1
46     1
47     1
48     1
49    -1
50     1
51     1
52     0
53    -1
54    -1
55     0
56    -1
57     1
58     0
59     0
60     1
61     1
62     1
63     0
64    -1
65     0
66    -1
67    -1
68     0
69     0
70     0
71     0
72     1
73     1
74     1
75    -1
76     0
77    -1
78    -1
79     0
80     1
81     0
82     0
83     0
84     1
85    -1
86    -1
87    -1
88    -1
89     0
90    -1
91     1
92     0
93    -1
94     0
95     1
96     0
97     1
98     1
99    -1
100    1
101   -1
102    1
103   -1
104    0
105   -1
106   -1
107   -1
108   -1
109    1
110    0
1

In [98]:
tweets_clean['sentiment'] = sentiments

In [99]:
tweets_clean.head()

,text,hour,minute,second,day,month,year,datetime,text_clean,sentiment
0,"""foda-se a eleição, eu quero ver o jogo porrraaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"" - Álvaro Dias",19,24,23,26,9,2018,2018-09-26 19:24:23,foder o eleição querer o jogar porrraaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa álvaro,0.0
1,MANO O DACIOLO É FODA GLÓRIA DEUS,19,12,45,26,9,2018,2018-09-26 19:12:45,manir o daciolo foder glória deus,1.0
2,"Porra Daciolo eu tinha até me emocionado com a declaração pra mamãe e esposa, agora tu mete essa de voto impresso meu consagrado? #DebateSBT",19,13,41,26,9,2018,2018-09-26 19:13:41,porra daciolo emocionar o declaração pra mamãe e esposo meter votar impresso consagrar debatesbt,-1.0
3,"Daciolo falou pouco, mas falou bonito...",18,37,45,26,9,2018,2018-09-26 18:37:45,daciolo falar falar bonito,1.0
4,"@morgalashnikova Eu n valorizei a grandeza desse candidato antes, mas agora eu percebi meu engano. 51 é só coisa boa, pinga e Daciolo",19,12,20,26,9,2018,2018-09-26 19:12:20,@morgalashnikova n valorizar o grandeza d candidatar antar perceber enganir 51 só pingar e daciolo,1.0


In [100]:
train = tweets_clean[tweets_clean['sentiment'].notnull()]

In [101]:
holdout = tweets_clean[tweets_clean['sentiment'].isnull()].drop('sentiment', axis=1)

In [102]:
train.to_csv('tweets_train.csv', index=False)
holdout.to_csv('tweets_holdout.csv', index=False)